<a href="https://colab.research.google.com/github/zoya-ivanova/BigData/blob/main/Spark_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install pyspark
!pip install pyspark >> None

In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

HW_1.Spark Apache
SQL & BigData

Найти самую длинную последовательность упорядоченных чисел в RDD и вывести её.

In [3]:
def find_long_sequence(rdd):
    # Преобразуем элементы RDD в целые числа и отсортируем их
    ordered_rdd = rdd.map(lambda x: int(x)).sortBy(lambda x: x).collect()

    # Создадим переменные для текущей последовательности и самой длинной последовательности
    current_sequence = []
    long_sequence = []

    for num in ordered_rdd:
        if not current_sequence or num == current_sequence[-1] + 1:
            current_sequence.append(num)
        else:
            if len(current_sequence) > len(long_sequence):
                long_sequence = current_sequence.copy()
            current_sequence = [num]

    # Сравниваем длину текущей последовательности с длиной самой длинной последовательности
    if len(current_sequence) > len(long_sequence):
        long_sequence = current_sequence

    return long_sequence

# Создание Spark-сессии
spark = SparkSession.builder.appName("longest_sequence").getOrCreate()

# Пример входного RDD с последовательностью чисел
data = [1, 2, 3, 5, 6, 7, 10, 11, 12, 13]
rdd = spark.sparkContext.parallelize(data)

# Находим самую длинную последовательность упорядоченных чисел в RDD
result = find_long_sequence(rdd)

print("Самая длинная последовательность:", result)

Самая длинная последовательность: [10, 11, 12, 13]


In [4]:
# Завершение Spark-сессии
spark.stop()

Создание нескольких Spark сессий

In [ ]:
from pyspark.sql import SparkSession

# Создание основной сессии Spark

spark = SparkSession.builder \
  .appName("MyApp") \
  .master("local[2]") \
  .config("spark.executor.memory", "2g") \
  .getOrCreate()

# Создание дополнительной сессии Spark с другой конфигурацией

spark2 = spark.newSession() \
  .config("spark.executor.memory", "1g") \
  .getOrCreate()

Литература:
- Изучаем Spark, молниеносный анализ данных
- Spark in Action

In [5]:
# 1/ Найти среднее значение элементов в RDD

sc = SparkContext("local", "Average RDD")
rdd = sc.parallelize([1, 2, 3, 4, 5, 6, 7])
mean_value = rdd.mean()
print("Среднее значение элементов в RDD: ", mean_value)

Среднее значение элементов в RDD:  4.0


In [5]:
active_session = SparkSession.active
print(active_session)

<bound method SparkSession.active of <class 'pyspark.sql.session.SparkSession'>>


In [7]:
sc.stop()

In [8]:
# 2/ Найти наибольший элемент в RDD

sc = SparkContext("local", "Max RDD")
rdd = sc.parallelize([100, 25, 30, 40, 55, 63, 78])
max_value = rdd.mean()
print("Среднее значение элементов в RDD: ", max_value)

Среднее значение элементов в RDD:  55.857142857142854


In [9]:
sc.stop()

In [10]:
# 3/ Посчитать количество элементов, удовлетворяющих условию

sc = SparkContext("local", "Filter RDD")

rdd = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
filtered_rdd = rdd.filter(lambda x: x > 5)
count = filtered_rdd.count()
print("Количество элементов, больших 5:", count)

Количество элементов, больших 5: 5


In [11]:
sc.stop()

In [12]:
# 4/ Дан набор данных с инф о продажах товаров (товар, магазин, кол).
# Необходимо сгруппировать данные по товару и
# найти суммарное количество проданных товаров по каждому товару

sc = SparkContext("local", "advanced")
data = [("apple", "store1", 10), ("apple", "store2", 15),
        ("banana", "store1", 20), ("banana", "store2", 25),
        ("peach", "store1", 5), ("peach", "store2", 10),
        ("peach", "store3", 25),]

rdd = sc.parallelize(data)

grouped_rdd = rdd.map(lambda x: (x[0], x[2])).reduceByKey(lambda a, b: a + b)
grouped_rdd.collect()

[('apple', 25), ('banana', 45), ('peach', 40)]

In [13]:
sc.stop()

In [14]:
# 5/ Дан набор данных с инф о продажах товаров (магазин, товар, кол, цена).
# Необходимо найти общую выручку от продаж каждого товара в каждом магазине

sc = SparkContext("local", "advanced")
data = [("store1", "apple", 10, 2), ("store2", "apple", 15, 2.5),
        ("store1", "banana", 20, 1.5), ("store2", "banana", 25, 1.8),
        ("store1", "watermelon", 3, 5), ("store2", "watermelon", 2, 4)]

rdd = sc.parallelize(data)

revenue_rdd = rdd.map(lambda x: ((x[0], x[1]), x[2]*x[3])).reduceByKey(lambda a, b: a + b)
revenue_rdd.collect()

[(('store1', 'apple'), 20),
 (('store2', 'apple'), 37.5),
 (('store1', 'banana'), 30.0),
 (('store2', 'banana'), 45.0),
 (('store1', 'watermelon'), 15),
 (('store2', 'watermelon'), 8)]

In [31]:
sc.stop()

In [32]:
# 6/ Даны 2 набора данных: 1 - с инф о продажах (товар, кол) и 2 - с инфо о цене товара (товар, цена).
# Необходимо объеденить данные и найти общую выручку от продаж каждого товара

sc = SparkContext("local", "advanced")
sales_data = [("apple", 10), ("banana", 20), ("apple", 15), ("banana", 20), ("peach", 15), ("peach", 25), ("watermelon", 5), ("watermelon", 10)]
price_data = [("apple", 2), ("banana", 1.5), ("peach", 2.5), ("watermelon", 3.5)]

sales_rdd = sc.parallelize(sales_data)
price_rdd = sc.parallelize(price_data)

joined_rdd = sales_rdd.join(price_rdd)

revenue_rdd = joined_rdd.map(lambda x: (x[0], x[1][0] * x[1][1]))
revenue_rdd.collect()


[('banana', 30.0),
 ('banana', 30.0),
 ('peach', 37.5),
 ('peach', 62.5),
 ('watermelon', 17.5),
 ('watermelon', 35.0),
 ('apple', 20),
 ('apple', 30)]

In [44]:
sc.stop()

In [22]:
# 7/ Поиск самого длинного слова в RDD, выводит первое попавшее по длине!!

sc = SparkContext("local", "RDD tasks")

data = ['приложение', 'Яблоко', 'Спарк', 'путеводитель','Метрополитен','Анализ','Солнце','питон','снег','рынок']
rdd = sc.parallelize(data)
longest_word = rdd.max(key=lambda x: len(x))
print("Самое длинное слово: ", longest_word)

Самое длинное слово:  путеводитель


In [43]:
sc.stop()

In [ ]:
# 8/ Фильтрация слов по длине в RDD

sc = SparkContext("local", "RDD tasks")

data = ['приложение', 'Яблоко', 'Спарк', 'путеводитель','Метрополитен','Анализ','Солнце','питон','снег','рынок']
rdd = sc.parallelize(data)
filtered_rdd = rdd.max(key=lambda x: len(x) > 6)
print("Самое длинное слово: ", filtered_rdd.collect())

In [21]:
sc.stop()

In [ ]:
# 9/ Подсчет количества уникальных слов в RDD

sc = SparkContext("local", "RDD tasks")

data = ['Приложение', 'Яблоко', 'Спарк', 'Путеводитель','Метрополитен','Анализ','Солнце','Питон',
        'Снег', 'Рынок', 'Яблоко', 'Путеводитель','Анализ']
rdd = sc.parallelize(data)
unique_words_count = rdd.distinct().count()
print("Количество уникальных слов:", unique_words_count())

In [38]:
sc.stop()

In [20]:
# 10/ Преобразование всех слов в RDD в верхний регистр

sc = SparkContext("local", "RDD tasks")
data = ['приложение', 'Яблоко', 'Спарк', 'путеводитель','Метрополитен','Анализ','Солнце','питон','снег','рынок']
rdd = sc.parallelize(data)
upper_rdd = rdd.map(lambda x: x.upper())
print("Слова в верхнем регистре: ", upper_rdd.collect())

Слова в верхнем регистре:  ['ПРИЛОЖЕНИЕ', 'ЯБЛОКО', 'СПАРК', 'ПУТЕВОДИТЕЛЬ', 'МЕТРОПОЛИТЕН', 'АНАЛИЗ', 'СОЛНЦЕ', 'ПИТОН', 'СНЕГ', 'РЫНОК']


In [13]:
sc.stop()

In [14]:
# 11/ Найти средний возраст пользователей по их покупкам и вывести топ -5 самых молодых

sc = SparkContext("local", "RDD tasks")
user_purchase_rdd = sc.parallelize([(1, 25), (2, 30), (3, 20), (4, 35), (5, 28), (6, 22)])
user_age_total = user_purchase_rdd.mapValues(lambda age: (age, 1)).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])).mapValues(lambda v: v[0] / v[1])
youngest_users = user_age_total.sortBy(lambda x: x[1]).take(5)
for user_id, avg_age in youngest_users:
  print(user_id, avg_age)

3 20.0
6 22.0
1 25.0
5 28.0
2 30.0


In [15]:
sc.stop()

In [16]:
# 12/ Найти среднюю цену товара в каждой категории и вывести результат в формате категория-средняя цена
sc = SparkContext("local", "ProductPraice")
product_rdd = sc.parallelize([(1, "A", 100), (2, "B", 150), (3, "A", 120), (4, "C", 200), (5, "B", 130), (6, "C", 140)])
category_total_price = product_rdd.map(lambda x: (x[1], (x[2], 1))).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])).mapValues(lambda v: v[0] / v[1])
for category, avg_price in category_total_price.collect():
  print(category, avg_price)

A 110.0
B 140.0
C 170.0


In [29]:
sc.stop()

In [ ]:
# 13/ Найти все пары чисел из RDD, сумма которых превышает 100 и вывести их
sc = SparkContext("local", "NumberPairs")
number_rdd = sc.parallelize([30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140])
number_pairs = number_rdd.cartesian(number_rdd).filter(lambda x: x[0] + [1] > 100).collect()
for pair in number_pairs:
  print(pair)